In [1]:
!pip install python-mnist

In [2]:
from mnist.loader import MNIST
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, AveragePooling2D

In [3]:
def Accuracy(y_true, y_pred):
  corr = 0
  for num,i in enumerate(y_true):
    if i==y_pred[num]:
      corr+=1
  corr /= len(y_true)
  return corr*100

In [4]:
def loadMNIST():
    mndata = MNIST('/content/MNIST')
    imagesTest, labelsTest = mndata.load_training()
    imagesTrain, labelsTrain = mndata.load_testing()
    return imagesTrain, labelsTrain, imagesTest, labelsTest
X_train, y_train, X_test, y_test = loadMNIST()
X_train = np.array(X_train)
X_test = np.array(X_test)

In [5]:
y_train_nn = np.array([ [0 for i in range(10)] for i in y_train ])
for i in y_train:
    y_train_nn[i] = 1
y_test_nn = np.array([ [0 for i in range(10)] for i in y_test ])
for i in y_test:
    y_test_nn[i] = 1

In [8]:
from sklearn.linear_model import LogisticRegression
def Linear_Classifier_and_Pairwise_Linear_Classifier():
    model = LogisticRegression(random_state=0, max_iter=1000)
    model = model.fit(X_train, y_train)
    return [Accuracy(model.predict(X_test), y_test)]

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
def Baseline_Nearest_Neighbor_Classifer():
    # k=3 was picked from the paper 
    # sklearn's defualt is 5
    model = KNeighborsClassifier(n_neighbors=3)
    model = model.fit(X_train, y_train)
    return [Accuracy(model.predict(X_test), y_test)]

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
def Principal_Component_Analysis_and_Polynomial_Classifier():
    # Convert 28*28 dim to 40
    # used to train a 2nd degree polynomial Classifer
    pca = PCA(n_components=40).fit(X_train)
    poly = PolynomialFeatures(2)
    X_train_updated = poly.fit_transform(pca.transform(X_train))
    X_test_updated = poly.transform(pca.transform(X_test))
    model = LogisticRegression(random_state=0, max_iter=1000)
    model = model.fit(X_train_updated, y_train)
    return [Accuracy(model.predict(X_test_updated), y_test)]

In [ ]:
def OneHidden_Layer_Neural_Network():
    ret = {}
    
    # 300 layer
    model_300 = Sequential()
    model_300.add(Dense(300, input_dim=784, activation="tanh"))
    model_300.add(Dense(10, activation="sigmoid"))
    model_300.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model_300.fit(X_train, y_train_nn,validation_data = (X_test,y_test_nn), epochs=100, batch_size=64)
    ret["28*28-300-10"] =  history.history['val_accuracy'][-1]

    # 1000 layer
    model_1000 = Sequential()
    model_1000.add(Dense(1000, input_dim=784, activation="tanh"))
    model_1000.add(Dense(10, activation="sigmoid"))
    model_1000.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model_1000.fit(X_train, y_train_nn,validation_data = (X_test,y_test_nn), epochs=100, batch_size=64)
    ret["28*28-1000-10"] =  history.history['val_accuracy'][-1]

    return ret

In [ ]:
def TwoHidden_Layer_Neural_Network():
    ret = {}
    
    # 300 layer
    model_300 = Sequential()
    model_300.add(Dense(300, input_dim=784, activation="tanh"))
    model_300.add(Dense(100, activation="tanh"))
    model_300.add(Dense(10, activation="sigmoid"))
    model_300.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model_300.fit(X_train, y_train_nn,validation_data = (X_test,y_test_nn), epochs=100, batch_size=64)
    ret["28*28-300-100-10"] =  max(history.history['val_accuracy'])*100

    # 1000 layer
    model_1000 = Sequential()
    model_1000.add(Dense(1000, input_dim=784, activation="tanh"))
    model_1000.add(Dense(150, activation="tanh"))
    model_1000.add(Dense(10, activation="sigmoid"))
    model_1000.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model_1000.fit(X_train, y_train_nn,validation_data = (X_test,y_test_nn), epochs=100, batch_size=64)
    ret["28*28-1000-150-10"] =  max(history.history['val_accuracy'])*100
    return ret

In [ ]:
models = ["Linear_Classifier_and_Pairwise_Linear_Classifier", "Baseline_Nearest_Neighbor_Classifer", 
          "Principal_Component_Analysis_and_Polynomial_Classifier", "OneHidden_Layer_Neural_Network",
          "TwoHidden_Layer_Neural_Network", "Lenet_1"]
names, acc = [], []
for i in models:
    temp = eval(i+"()")
    if len(temp)>1:
      for i in temp:
        names.append(i)
        acc.append(temp[i])
    else:
        names.append(" ".join(i.split("_")))
        acc.append(temp[0])
df = pd.DataFrame()
df["Models"] = names
df["Accuracies"] = acc

In [ ]:
def Lenet_1():
    # model
    model = Sequential()
    model.add(Conv2D(4, kernel_size=5, activation="tanh", input_shape=(28,28,1)))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
    model.add(Conv2D(12, kernel_size=5, activation="tanh"))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
    model.add(Flatten())
    model.add(Dense(10, activation="sigmoid"))
    # model.summary()

    # Compile, train and find accuracy
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train.reshape(len(X_train), 28, 28, 1), y_train_nn,
                        validation_data = (X_test.reshape(len(X_test), 28, 28, 1),y_test_nn), epochs=3, batch_size=64)
    return [max(history.history['val_accuracy'])*100]

In [16]:
def Lenet_4():
    # Pad the data to get 32*32 input dimensions
    X_train_pad = X_train.reshape(len(X_train), 28, 28)
    X_test_pad = X_test.reshape(len(X_test), 28, 28)
    X_train_pad = np.array([np.pad(i, (2)) for i in X_train_pad])
    X_test_pad = np.array([np.pad(i, (2)) for i in X_test_pad])

    # model
    model = Sequential()
    model.add(Conv2D(4, kernel_size=5, activation="tanh", input_shape=(32,32,1)))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
    model.add(Conv2D(16, kernel_size=5, activation="tanh"))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
    model.add(Flatten())
    model.add(Dense(120, activation="tanh"))
    model.add(Dense(10, activation="sigmoid"))
    # model.summary()

    # Compile, train and find accuracy
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train_pad, y_train_nn,
                        validation_data = (X_test_pad,y_test_nn), epochs=3, batch_size=64)
    return [max(history.history['val_accuracy'])*100]

In [17]:
Lenet_4()

Epoch 1/3
157/157 [==============================] - 18s 112ms/step - loss: 0.0239 - accuracy: 0.0011 - val_loss: 0.0040 - val_accuracy: 0.0000e+00
Epoch 2/3
157/157 [==============================] - 17s 111ms/step - loss: 0.0239 - accuracy: 0.0000e+00 - val_loss: 0.0040 - val_accuracy: 0.0000e+00
Epoch 3/3
157/157 [==============================] - 17s 112ms/step - loss: 0.0239 - accuracy: 0.0000e+00 - val_loss: 0.0040 - val_accuracy: 0.0000e+00


[0.0]